In [1]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load Data

In [2]:
event_data = pd.read_csv('MMADataset2023/ufc_event_data.csv')
fight_data = pd.read_csv('MMADataset2023/ufc_fight_data.csv')
fight_stat_data = pd.read_csv('MMADataset2023/ufc_fight_stat_data.csv')
fighter_data = pd.read_csv('MMADataset2023/ufc_fighter_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'MMADataset2023/ufc_event_data.csv'

In [3]:
print("Event Data:")
display(event_data.head())

print("Fight Data:")
display(fight_data.head())

print("Fight Stat Data:")
display(fight_stat_data.head())

print("Fighter Data:")
display(fighter_data.head())

Event Data:


NameError: name 'event_data' is not defined

## Explore the Data

In [173]:
print("Event Data Shape:", event_data.shape)
print("Fight Data Shape:", fight_data.shape)
print("Fight Stat Data Shape:", fight_stat_data.shape)
print("Fighter Data Shape:", fighter_data.shape)

Event Data Shape: (665, 7)
Fight Data Shape: (7218, 15)
Fight Stat Data Shape: (14436, 14)
Fighter Data Shape: (4107, 14)


### Null values and data types

In [174]:
event_data.info()
fight_data.info()
fight_stat_data.info()
fighter_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   event_id       665 non-null    int64 
 1   event_name     665 non-null    object
 2   event_date     665 non-null    object
 3   event_city     665 non-null    object
 4   event_state    616 non-null    object
 5   event_country  665 non-null    object
 6   event_url      665 non-null    object
dtypes: int64(1), object(6)
memory usage: 36.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7218 entries, 0 to 7217
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fight_id        7218 non-null   int64  
 1   event_id        7218 non-null   int64  
 2   referee         7186 non-null   object 
 3   f_1             7199 non-null   float64
 4   f_2             7205 non-null   float64
 5   winner          7203 non-null  

In [175]:
print(event_data.isnull().sum())

event_id          0
event_name        0
event_date        0
event_city        0
event_state      49
event_country     0
event_url         0
dtype: int64


In [176]:
print(fight_data.isnull().sum())

fight_id           0
event_id           0
referee           32
f_1               19
f_2               13
winner            15
num_rounds         0
title_fight        0
weight_class      13
gender             0
result             0
result_details    17
finish_round       0
finish_time        0
fight_url          0
dtype: int64


In [177]:
print(fight_stat_data.isnull().sum())

fight_stat_id          0
fight_id               0
fighter_id            32
knockdowns            42
total_strikes_att     42
total_strikes_succ    42
sig_strikes_att       42
sig_strikes_succ      42
takedown_att          42
takedown_succ         42
submission_att        42
reversals             42
ctrl_time             42
fight_url              0
dtype: int64


In [178]:
print(fighter_data.isnull().sum())

fighter_id               0
fighter_f_name           0
fighter_l_name          15
fighter_nickname      1857
fighter_height_cm      310
fighter_weight_lbs      87
fighter_reach_cm      1941
fighter_stance         834
fighter_dob            758
fighter_w                0
fighter_l                0
fighter_d                0
fighter_nc_dq         3625
fighter_url              0
dtype: int64


## Remove Duplicates

In [179]:
event_duplicates = event_data.duplicated().sum()
fight_duplicates = fight_data.duplicated().sum()
fight_stat_duplicates = fight_stat_data.duplicated().sum()
fighter_duplicates = fighter_data.duplicated().sum()

In [180]:
print("Number of duplicate rows in Event Data:", event_duplicates)
print("Number of duplicate rows in Fight Data:", fight_duplicates)
print("Number of duplicate rows in Fight Stat Data:", fight_stat_duplicates)
print("Number of duplicate rows in Fighter Data:", fighter_duplicates)

Number of duplicate rows in Event Data: 0
Number of duplicate rows in Fight Data: 0
Number of duplicate rows in Fight Stat Data: 0
Number of duplicate rows in Fighter Data: 0


In [181]:
event_data.drop_duplicates(inplace=True)
fight_data.drop_duplicates(inplace=True)
fight_stat_data.drop_duplicates(inplace=True)
fighter_data.drop_duplicates(inplace=True)

In [182]:
print("Updated Event Data Shape:", event_data.shape)
print("Updated Fight Data Shape:", fight_data.shape)
print("Updated Fight Stat Data Shape:", fight_stat_data.shape)
print("Updated Fighter Data Shape:", fighter_data.shape)

Updated Event Data Shape: (665, 7)
Updated Fight Data Shape: (7218, 15)
Updated Fight Stat Data Shape: (14436, 14)
Updated Fighter Data Shape: (4107, 14)


### There do not seem to be any duplicates but running it just in case.

# Assessment of critical vs non-critical values

### event_data: 
- Non-Critical:
     - event_state (49 missing) -> fill with "Unknown"
     
### fight_data: 
- Critical:
    - f_1 (19 missing) -> drop
    - f_2 (13 missing) -> drop
    - winner (15 missing) -> drop
    - weight_class (13 missing) -> drop
- Non-Critical: 
    - referee (32 missing) -> fill with "Unknown"
    - result_details (17 missing) -> fill with "N/A" 
        - "Specific details of how the fight ended, e.g. KO by elbows, split decision"

### fight_stat_data:
- Critical:
    - fighter_id (32 missing) -> drop (can't link stats without fighter since its a foreign key) 
    - total_strikes_att (42 missing) -> drop (Performance metrics)
    - total_strikes_succ (42 missing) -> drop
    - sig_strikes_att (42) missing) -> drop
    - sig_strikes_succ (42 missing) -> drop
- Non-Critical: 
    - knockdowns (42 missing) -> fill with 0
    - submission_att, reversals, ctrl_time (42 missing) -> fill with 0
    
### fighter_data:        
- Critical:
    - fighter_stance (834 missing) -> drop
    - fighter_dob (758 missing) -> drop
    - fighter_height_cm (310 missing) -> drop (maybe fill with median)
    - fighter_weight_lbs (87 missing) -> drop (maybe fill with median)
    - fighter_reach_cm (1941 missing) -> drop (maybe fill with median)
- Non-Critical:
    - fighter_l_name (15 missing) -> fill with "Unknown"
    - fighter_nickname (1857 missing) -> fill with "Unknown"
    - fighter_nc_dq (3625 missing) -> fill with 0 

### Drop rows with critical missing values

In [183]:
fight_data = fight_data.dropna(subset=['f_1', 'f_2', 'winner', 'weight_class'])


fight_stat_data = fight_stat_data.dropna(subset=[
    'fighter_id', 'total_strikes_att', 'total_strikes_succ',
    'sig_strikes_att', 'sig_strikes_succ'
])


fighter_data = fighter_data.dropna(subset=['fighter_stance', 'fighter_dob'])

### Fill non-critical missing values

In [184]:
event_data['event_state'].fillna('Unknown', inplace=True)


fight_data['referee'].fillna('Unknown', inplace=True)
fight_data['result_details'].fillna('N/A', inplace=True)


fight_stat_data[['knockdowns', 'submission_att', 'reversals', 'ctrl_time']] = \
    fight_stat_data[['knockdowns', 'submission_att', 'reversals', 'ctrl_time']].fillna(0)


fighter_data['fighter_l_name'].fillna('Unknown', inplace=True)
fighter_data['fighter_nickname'].fillna('Unknown', inplace=True)
fighter_data['fighter_nc_dq'].fillna(0, inplace=True)

### If we want to fill with median: 
fighter_data['fighter_height_cm'].fillna(fighter_data['fighter_height_cm'].median(), inplace=True)
fighter_data['fighter_weight_lbs'].fillna(fighter_data['fighter_weight_lbs'].median(), inplace=True)
fighter_data['fighter_reach_cm'].fillna(fighter_data['fighter_reach_cm'].median(), inplace=True)


## Standardize Column Names

Make all column names lowercase and replace spaces with _ 

In [185]:
event_data.columns = event_data.columns.str.lower().str.replace(' ', '_')
fight_data.columns = fight_data.columns.str.lower().str.replace(' ', '_')
fight_stat_data.columns = fight_stat_data.columns.str.lower().str.replace(' ', '_')
fighter_data.columns = fighter_data.columns.str.lower().str.replace(' ', '_')

In [186]:
print("Event Data Columns:", list(event_data.columns))
print("Fight Data Columns:", list(fight_data.columns))
print("Fight Stat Data Columns:", list(fight_stat_data.columns))
print("Fighter Data Columns:", list(fighter_data.columns))

Event Data Columns: ['event_id', 'event_name', 'event_date', 'event_city', 'event_state', 'event_country', 'event_url']
Fight Data Columns: ['fight_id', 'event_id', 'referee', 'f_1', 'f_2', 'winner', 'num_rounds', 'title_fight', 'weight_class', 'gender', 'result', 'result_details', 'finish_round', 'finish_time', 'fight_url']
Fight Stat Data Columns: ['fight_stat_id', 'fight_id', 'fighter_id', 'knockdowns', 'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att', 'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att', 'reversals', 'ctrl_time', 'fight_url']
Fighter Data Columns: ['fighter_id', 'fighter_f_name', 'fighter_l_name', 'fighter_nickname', 'fighter_height_cm', 'fighter_weight_lbs', 'fighter_reach_cm', 'fighter_stance', 'fighter_dob', 'fighter_w', 'fighter_l', 'fighter_d', 'fighter_nc_dq', 'fighter_url']


### Convert Data Types

In [187]:
if 'event_date' in event_data.columns:
    event_data['event_date'] = pd.to_datetime(event_data['event_date'], errors='coerce')
    
    
fight_data['f_1'] = fight_data['f_1'].astype(int, errors='ignore')
fight_data['f_2'] = fight_data['f_2'].astype(int, errors='ignore')
fight_data['fight_id'] = fight_data['fight_id'].astype(int, errors='ignore')
fight_data['num_rounds'] = pd.to_numeric(fight_data['num_rounds'], errors='coerce').astype('Int64')


fighter_data['fighter_height_cm'] = fighter_data['fighter_height_cm'].astype(float, errors='ignore')
fighter_data['fighter_weight_lbs'] = fighter_data['fighter_weight_lbs'].astype(float, errors='ignore')
fighter_data['fighter_reach_cm'] = fighter_data['fighter_reach_cm'].astype(float, errors='ignore')
if 'fighter_dob' in fighter_data.columns:
    fighter_data['fighter_dob'] = pd.to_datetime(fighter_data['fighter_dob'], errors='coerce')
    
    
fight_stat_data['total_strikes_att'] = fight_stat_data['total_strikes_att'].astype(int, errors='ignore')
fight_stat_data['total_strikes_succ'] = fight_stat_data['total_strikes_succ'].astype(int, errors='ignore')
fight_stat_data['sig_strikes_att'] = fight_stat_data['sig_strikes_att'].astype(int, errors='ignore')
fight_stat_data['sig_strikes_succ'] = fight_stat_data['sig_strikes_succ'].astype(int, errors='ignore')
fight_stat_data['knockdowns'] = fight_stat_data['knockdowns'].astype(int, errors='ignore')
fight_stat_data['submission_att'] = fight_stat_data['submission_att'].astype(int, errors='ignore')
fight_stat_data['reversals'] = fight_stat_data['reversals'].astype(int, errors='ignore')
fight_stat_data['ctrl_time'] = fight_stat_data['ctrl_time'].astype(int, errors='ignore')
fight_stat_data['ctrl_time'] = pd.to_numeric(fight_stat_data['ctrl_time'], errors='coerce').astype('Int64')

In [188]:
print("Event Data Types:")
print(event_data.dtypes)

print("\nFight Data Types:")
print(fight_data.dtypes)

print("\nFight Stat Data Types:")
print(fight_stat_data.dtypes)

print("\nFighter Data Types:")
print(fighter_data.dtypes)

Event Data Types:
event_id                  int64
event_name               object
event_date       datetime64[ns]
event_city               object
event_state              object
event_country            object
event_url                object
dtype: object

Fight Data Types:
fight_id            int64
event_id            int64
referee            object
f_1                 int64
f_2                 int64
winner            float64
num_rounds          Int64
title_fight        object
weight_class       object
gender             object
result             object
result_details     object
finish_round        int64
finish_time        object
fight_url          object
dtype: object

Fight Stat Data Types:
fight_stat_id           int64
fight_id                int64
fighter_id            float64
knockdowns              int64
total_strikes_att       int64
total_strikes_succ      int64
sig_strikes_att         int64
sig_strikes_succ        int64
takedown_att          float64
takedown_succ         float

## Drop irrelevant Columns

### Relevant:
1. ufc_events:
    - event_id
    - event_date
    - event_country

2. ufc_fights:
    - fight_id
    - event_id
    - f_1
    - f_2
    - winner
    - num_rounds
    - title_fight
    - weight_class
    - gender
    - result
    - result_details
    - finish_round
    - finish_time

3. ufc_fight_stats:
    - fight_stat_id
    - fight_id
    - fighter_id
    - knockdowns
    - total_strikes_att
    - total_strikes_succ
    - sig_strikes_att
    - sig_strikes_succ
    - takedown_att
    - takedown_succ
    - submission_att
    - reversals
    - ctrl_time
    - fighter_age
    - winner

4. ufc_fighters:
    - fighter_id
    - fighter_height_cm
    - fighter_weight_lbs
    - fighter_reach_cm
    - fighter_stance
    - fighter_dob
    - fighter_w
    - fighter_l
    - fighter_d
    - fighter_nc_dq

### Irrelevant:
1. ufc_events:
    - event_name
    - event_city
    - event_state
    - event_url

2. ufc_fights:
    - referee
    - fight_url

3. ufc_fight_stats:
    - None

4. ufc_fighters:
    - fighter_f_name
    - fighter_l_name
    - fighter_nickname
    - fighter_url

In [189]:
event_columns_to_drop = ['event_name', 'event_city', 'event_state', 'event_url']
fight_columns_to_drop = ['referee', 'fight_url']
fighter_columns_to_drop = ['fighter_f_name', 'fighter_l_name', 'fighter_nickname', 'fighter_url']

event_data.drop(columns=[col for col in event_columns_to_drop if col in event_data.columns], inplace=True)
fight_data.drop(columns=[col for col in fight_columns_to_drop if col in fight_data.columns], inplace=True)
fighter_data.drop(columns=[col for col in fighter_columns_to_drop if col in fighter_data.columns], inplace=True)

{
    "event_data_columns": event_data.columns.tolist(),
    "fight_data_columns": fight_data.columns.tolist(),
    "fight_stat_data_columns": fight_stat_data.columns.tolist(),
    "fighter_data_columns": fighter_data.columns.tolist()
}

{'event_data_columns': ['event_id', 'event_date', 'event_country'],
 'fight_data_columns': ['fight_id',
  'event_id',
  'f_1',
  'f_2',
  'winner',
  'num_rounds',
  'title_fight',
  'weight_class',
  'gender',
  'result',
  'result_details',
  'finish_round',
  'finish_time'],
 'fight_stat_data_columns': ['fight_stat_id',
  'fight_id',
  'fighter_id',
  'knockdowns',
  'total_strikes_att',
  'total_strikes_succ',
  'sig_strikes_att',
  'sig_strikes_succ',
  'takedown_att',
  'takedown_succ',
  'submission_att',
  'reversals',
  'ctrl_time',
  'fight_url'],
 'fighter_data_columns': ['fighter_id',
  'fighter_height_cm',
  'fighter_weight_lbs',
  'fighter_reach_cm',
  'fighter_stance',
  'fighter_dob',
  'fighter_w',
  'fighter_l',
  'fighter_d',
  'fighter_nc_dq']}

## Key Consistency Across Datasets

1. Verify fight_id values in fight_stat_data exist in fight_data 
    - remove records in fight_stat_data that don't have entries in fight_data
2. all fighter_id values in fight_stat_data exists in fighter_data
    - handle missing fighter_id records in fighter_data 
3. remove duplicates based on primary keys (event_id, fight_id, fight_stat_id, fighter_id) 
4. Foreign key dependencies:
    - event_id in fight_data exists in event_data
    - fight_id in fight_stat_data has corresponding entries in fight_data
    - fighter_id in fight_stat_data maps correctly to fighters in fighter_data
5. Data type consistency across fight_id, fighter_id, and event_id
6. Missing key Relations
    - if fighter_id or fight_id is missing drop those rows from dependent datatset (fight_stat_data) 

In [190]:
valid_fight_ids = fight_data['fight_id'].unique()
fight_stat_data = fight_stat_data[fight_stat_data['fight_id'].isin(valid_fight_ids)]

valid_fighter_ids = fighter_data['fighter_id'].unique()
fight_stat_data = fight_stat_data[fight_stat_data['fighter_id'].isin(valid_fighter_ids)]

valid_event_ids = event_data['event_id'].unique()
fight_data = fight_data[fight_data['event_id'].isin(valid_event_ids)]

event_data.drop_duplicates(subset=['event_id'], inplace=True)
fight_data.drop_duplicates(subset=['fight_id'], inplace=True)
fight_stat_data.drop_duplicates(subset=['fight_stat_id'], inplace=True)
fighter_data.drop_duplicates(subset=['fighter_id'], inplace=True)

fight_data['fight_id'] = fight_data['fight_id'].astype(int)
fight_stat_data['fight_id'] = fight_stat_data['fight_id'].astype(int)
fighter_data['fighter_id'] = fighter_data['fighter_id'].astype(int)


### Outlier Detection

In [194]:
fight_stat_columns = ['knockdowns', 'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att', 
                      'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att', 
                      'reversals', 'ctrl_time']
fighter_data_columns = ['fighter_height_cm', 'fighter_weight_lbs', 'fighter_reach_cm', 
                        'fighter_w', 'fighter_l', 'fighter_d', 'fighter_nc_dq']
fight_data_columns = ['num_rounds', 'finish_round', 'finish_time']

def print_column_averages(df, columns, dataset_name):
    averages = df[columns].mean(numeric_only=True)  
    print(f"Average values for {dataset_name}:")
    print(averages, "\n")
    
print_column_averages(fight_stat_data, fight_stat_columns, "fight_stat_data")
print_column_averages(fighter_data, fighter_data_columns, "fighter_data")
print_column_averages(fight_data, fight_data_columns, "fight_data")

Average values for fight_stat_data:
knockdowns              0.218572
total_strikes_att     102.333452
total_strikes_succ     54.221351
sig_strikes_att        81.891819
sig_strikes_succ       36.236958
takedown_att              2.8249
takedown_succ           1.069128
submission_att          0.394598
reversals               0.135048
ctrl_time                   <NA>
dtype: object 

Average values for fighter_data:
fighter_height_cm     178.156597
fighter_weight_lbs    168.623438
fighter_reach_cm      181.846719
fighter_w              14.122256
fighter_l               5.964539
fighter_d               0.270854
fighter_nc_dq           0.153664
dtype: float64 

Average values for fight_data:
num_rounds      3.130021
finish_round    2.333333
dtype: float64 



In [204]:

def remove_outliers(df, columns):
    for col in columns:
        if df[col].dtype in [np.float64, np.int64]:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

# Remove outliers from fight_stat_data, fighter_data, and fight_data
fight_stat_columns = ['knockdowns', 'total_strikes_att', 'total_strikes_succ', 'sig_strikes_att', 
                      'sig_strikes_succ', 'takedown_att', 'takedown_succ', 'submission_att', 
                      'reversals', 'ctrl_time']
fighter_data_columns = ['fighter_height_cm', 'fighter_weight_lbs', 'fighter_reach_cm', 
                        'fighter_w', 'fighter_l', 'fighter_d', 'fighter_nc_dq']
fight_data_columns = ['num_rounds', 'finish_round', 'finish_time']

df_fight_stat_data_no_outliers = remove_outliers(fight_stat_data, fight_stat_columns)
df_fighter_data_no_outliers = remove_outliers(fighter_data, fighter_data_columns)
df_fight_data_no_outliers = remove_outliers(fight_data, fight_data_columns)

# Print average values after removing outliers
print_column_averages(df_fight_stat_data_no_outliers, fight_stat_columns, "fight_stat_data (no outliers)")
print_column_averages(df_fighter_data_no_outliers, fighter_data_columns, "fighter_data (no outliers)")
print_column_averages(df_fight_data_no_outliers, fight_data_columns, "fight_data (no outliers)")


Average values for fight_stat_data (no outliers):
knockdowns                  0.0
total_strikes_att     90.698296
total_strikes_succ    46.003915
sig_strikes_att        73.55562
sig_strikes_succ      30.933211
takedown_att           2.457393
takedown_succ           0.86596
submission_att         0.272225
reversals                   0.0
ctrl_time                  <NA>
dtype: object 

Average values for fighter_data (no outliers):
fighter_height_cm     176.717575
fighter_weight_lbs    156.381436
fighter_reach_cm      181.004255
fighter_w              12.649729
fighter_l               4.587398
fighter_d               0.000000
fighter_nc_dq           0.000000
dtype: float64 

Average values for fight_data (no outliers):
num_rounds      3.130021
finish_round    2.333333
dtype: float64 



## Save Cleaned Dataset

In [205]:
event_data.to_csv('Cleaned Dataset/cleaned_ufc_event_data.csv', index=False)
fight_data.to_csv('Cleaned Dataset/cleaned_ufc_fight_data.csv', index=False)
fight_stat_data.to_csv('Cleaned Dataset/cleaned_ufc_fight_stat_data.csv', index=False)
fighter_data.to_csv('Cleaned Dataset/cleaned_ufc_fighter_data.csv', index=False)